# Working with NBA live data...
Libraries supporting live data do not include support for Pandas. Note: Any call to `{endpoint}.{Class}()` will perform a request. Example: `scoreboard.ScoreBoard()`. In order to avoid multiple requests, set `{endpoint}.{Class}()` to a variable. See sample code below.


## Today's Score Board


In [1]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2021-01-17
0022000196: Knicks vs. Celtics @ 2021-01-17 13:00:00-05:00
0022000197: Cavaliers vs. Wizards @ 2021-01-17 14:00:00-05:00
0022000198: Bulls vs. Mavericks @ 2021-01-17 15:00:00-05:00
0022000199: 76ers vs. Thunder @ 2021-01-17 19:00:00-05:00
0022000200: Jazz vs. Nuggets @ 2021-01-17 20:00:00-05:00
0022000201: Pelicans vs. Kings @ 2021-01-17 21:00:00-05:00
0022000202: Pacers vs. Clippers @ 2021-01-17 22:00:00-05:00


## Box Score

In [1]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022000196') 

In [5]:
#Game Details
box.game_info.get_dict()

{'gameId': '0022000196',
 'gameTimeLocal': '2021-01-17T13:00:00-05:00',
 'gameTimeUTC': '2021-01-17T18:00:00Z',
 'gameTimeHome': '2021-01-17T13:00:00-05:00',
 'gameTimeAway': '2021-01-17T13:00:00-05:00',
 'gameEt': '2021-01-17T13:00:00-05:00',
 'duration': 134,
 'gameCode': '20210117/NYKBOS',
 'gameStatusText': 'Final',
 'gameStatus': 3,
 'regulationPeriods': 4,
 'period': 4,
 'gameClock': 'PT00M00.00S',
 'attendance': 0,
 'sellout': '0'}

In [4]:
#Arena
box.arena.get_dict()

{'arenaId': 17,
 'arenaName': 'TD Garden',
 'arenaCity': 'Boston',
 'arenaState': 'MA',
 'arenaCountry': 'US',
 'arenaTimezone': 'America/New_York'}

In [6]:
#Officials
box.officials.get_dict()

[{'personId': 2882,
  'name': 'Sean Wright',
  'nameI': 'S. Wright',
  'firstName': 'Sean',
  'familyName': 'Wright',
  'jerseyNum': '4',
  'assignment': 'OFFICIAL1'},
 {'personId': 201638,
  'name': 'Brent Barnaky',
  'nameI': 'B. Barnaky',
  'firstName': 'Brent',
  'familyName': 'Barnaky',
  'jerseyNum': '36',
  'assignment': 'OFFICIAL2'},
 {'personId': 1627524,
  'name': 'Nate Green',
  'nameI': 'N. Green',
  'firstName': 'Nate',
  'familyName': 'Green',
  'jerseyNum': '65',
  'assignment': 'OFFICIAL3'}]

In [32]:
#homeTeam & #awayTeam have the identicial data structure.
players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in players:
    print(f.format(player_id=player['personId'],name=player['name'],points=player['statistics']['points']))

203493: Reggie Bullock: 11 PTS
203944: Julius Randle: 20 PTS
1629011: Mitchell Robinson: 8 PTS
1629628: RJ Barrett: 19 PTS
203901: Elfrid Payton: 9 PTS
203457: Nerlens Noel: 5 PTS
203085: Austin Rivers: 0 PTS
1630193: Immanuel Quickley: 17 PTS
1630167: Obi Toppin: 12 PTS
1628995: Kevin Knox II: 3 PTS
1628372: Dennis Smith Jr.: 1 PTS
1629033: Theo Pinson: 0 PTS
1629649: Ignas Brazdeikis: 0 PTS
201959: Taj Gibson: 0 PTS
1629607: Jared Harper: 0 PTS
202692: Alec Burks: 0 PTS
1628373: Frank Ntilikina: 0 PTS


In [2]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022000196') 
box.home_team_stats.get_dict()
box.away_team_stats.get_dict()

{'teamId': 1610612752,
 'teamName': 'Knicks',
 'teamCity': 'New York',
 'teamTricode': 'NYK',
 'score': 105,
 'inBonus': '1',
 'timeoutsRemaining': 2,
 'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 28},
  {'period': 2, 'periodType': 'REGULAR', 'score': 20},
  {'period': 3, 'periodType': 'REGULAR', 'score': 27},
  {'period': 4, 'periodType': 'REGULAR', 'score': 30}],
 'statistics': {'assists': 22,
  'assistsTurnoverRatio': 1.29411764705882,
  'benchPoints': 38,
  'biggestLead': 37,
  'biggestLeadScore': '101-64',
  'biggestScoringRun': 12,
  'biggestScoringRunScore': '101-64',
  'blocks': 7,
  'blocksReceived': 2,
  'fastBreakPointsAttempted': 3,
  'fastBreakPointsMade': 2,
  'fastBreakPointsPercentage': 0.666666666666666,
  'fieldGoalsAttempted': 80,
  'fieldGoalsEffectiveAdjusted': 0.55,
  'fieldGoalsMade': 38,
  'fieldGoalsPercentage': 0.475,
  'foulsOffensive': 3,
  'foulsDrawn': 22,
  'foulsPersonal': 28,
  'foulsTeam': 25,
  'foulsTechnical': 0,
  'foulsTeamTechnical

In [2]:
# Query nba.live.endpoints for the score board of GameID 002200011 = IND vs NYK
from nba_api.live.nba.endpoints import playbyplay
print(playbyplay.PlayByPlay('0022000180').get_response())


{"meta": {"version": 1, "code": 200, "request": "http://nba.cloud/games/0022000180/playbyplay?Format=json", "time": "2021-01-15 23:48:58.906160"}, "game": {"gameId": "0022000180", "actions": [{"actionNumber": 2, "clock": "PT12M00.00S", "timeActual": "2021-01-16T00:40:29.8Z", "period": 1, "periodType": "REGULAR", "actionType": "period", "subType": "start", "qualifiers": [], "personId": 0, "x": null, "y": null, "possession": 0, "scoreHome": "0", "scoreAway": "0", "edited": "2021-01-16T00:40:29Z", "orderNumber": 20000, "xLegacy": null, "yLegacy": null, "isFieldGoal": 0, "side": null, "description": "Period Start", "personIdsFilter": []}, {"actionNumber": 4, "clock": "PT11M58.00S", "timeActual": "2021-01-16T00:40:31.3Z", "period": 1, "periodType": "REGULAR", "teamId": 1610612738, "teamTricode": "BOS", "actionType": "jumpball", "subType": "recovered", "descriptor": "startperiod", "qualifiers": [], "personId": 1629684, "x": null, "y": null, "possession": 1610612738, "scoreHome": "0", "scoreA

In [49]:
# Query nba.live.endpoints for the play by play of GameID 002200011 = IND vs NYK
from nba_api.live.nba.endpoints import playbyplay
f = "{k_key}:{v_type}"
e = "{k_key}:<{v_value}>"

actions = playbyplay.PlayByPlay('0022000180').get_dict()['game']['actions']
action_keys = []
no_actions = 0
no_keys = 0
for actions in actions:
    no_actions = no_actions + 1
    for key in actions.keys():
        value = f.format(k_key=key,v_type=type(actions[key]))
        if value not in action_keys :
            no_keys = no_keys + 1
            action_keys.append(value)
print(sorted(action_keys))
print(no_actions)
print(no_keys)

["actionNumber:<class 'int'>", "actionType:<class 'str'>", "assistPersonId:<class 'int'>", "assistPlayerNameInitial:<class 'str'>", "assistTotal:<class 'int'>", "blockPersonId:<class 'int'>", "blockPlayerName:<class 'str'>", "clock:<class 'str'>", "description:<class 'str'>", "descriptor:<class 'str'>", "edited:<class 'str'>", "foulDrawnPersonId:<class 'int'>", "foulDrawnPlayerName:<class 'str'>", "foulPersonalTotal:<class 'int'>", "foulTechnicalTotal:<class 'int'>", "isFieldGoal:<class 'int'>", "jumpBallLostPersonId:<class 'int'>", "jumpBallLostPlayerName:<class 'str'>", "jumpBallRecoverdPersonId:<class 'int'>", "jumpBallRecoveredName:<class 'str'>", "jumpBallWonPersonId:<class 'int'>", "jumpBallWonPlayerName:<class 'str'>", "officialId:<class 'int'>", "orderNumber:<class 'int'>", "period:<class 'int'>", "periodType:<class 'str'>", "personId:<class 'int'>", "personIdsFilter:<class 'list'>", "playerName:<class 'str'>", "playerNameI:<class 'str'>", "pointsTotal:<class 'int'>", "possessi